In [2]:
import os

os.chdir("../")

In [1]:
%pwd

'd:\\Cdac_ML\\Assignments\\Assignment_3\\research'

In [3]:
#entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    criterion: str
    max_depth: int
    min_samples_split: int
    min_samples_leaf: int
    min_weight_fraction_leaf: float
    max_features: str
    max_leaf_nodes: int
    min_impurity_decrease: float
    bootstrap: str
    oob_score: str
    n_jobs: int
    random_state: int
    verbose: int
    warm_start: str
    class_weight: int
    ccp_alpha: float
    max_samples: int
    monotonic_cst: int
    target_column: str
    params : any

In [4]:
#configuration manager in src config

from src.Assignment_3.constants import *
from src.Assignment_3.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = Config_yaml_path,
        params_filepath = params_yaml_path,
        schema_filepath = schema_yaml_path):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            criterion = params.criterion,
            max_depth = params.max_depth,
            min_samples_split = params.min_samples_split,
            min_samples_leaf = params.min_samples_leaf,
            min_weight_fraction_leaf = params.min_weight_fraction_leaf,
            max_features = params.max_features,
            max_leaf_nodes = params.max_leaf_nodes,
            min_impurity_decrease = params.min_impurity_decrease,
            bootstrap = params.bootstrap,
            oob_score = params.oob_score,
            n_jobs = params.n_jobs,
            random_state = params.random_state,
            verbose = params.verbose,
            warm_start = params.warm_start,
            class_weight = params.class_weight,
            ccp_alpha = params.ccp_alpha,
            max_samples = params.max_samples,
            monotonic_cst = params.monotonic_cst,
            target_column = schema.name,
            params=self.params

            
        )

        return model_trainer_config

In [5]:
import pandas as pd
import os
from src.Assignment_3 import logger
from sklearn.ensemble import RandomForestClassifier
import joblib

In [6]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)

        # ✅ FIX: use 1D Series
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        # ✅ FIX: correct params reference
        params = self.config.params.RandomForestClassifier


        rf = RandomForestClassifier(
            n_estimators = params.n_estimators,
            criterion = params.criterion,
            max_depth = params.max_depth,
            min_samples_split = params.min_samples_split,
            min_samples_leaf = params.min_samples_leaf,
            min_weight_fraction_leaf = params.min_weight_fraction_leaf,
            max_features = params.max_features,
            max_leaf_nodes = params.max_leaf_nodes,
            min_impurity_decrease = params.min_impurity_decrease,
            bootstrap = params.bootstrap,
            oob_score = params.oob_score,
            n_jobs = params.n_jobs,
            random_state = params.random_state,
            verbose = params.verbose,
            warm_start = params.warm_start,
            class_weight = params.class_weight,
            ccp_alpha = params.ccp_alpha,
            max_samples = params.max_samples,
            monotonic_cst = params.monotonic_cst,
        )

        # ✅ FIX: y must be 1D
        rf.fit(train_x, train_y)

        joblib.dump(rf, os.path.join(self.config.root_dir, self.config.model_name))


In [7]:
# pipeline

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_tr_config = ModelTrainer(config=model_trainer_config)
    model_tr_config.train()
except Exception as e:
    raise e

[2025-11-06 23:42:59,354: INFO: common: yaml file: D:\Cdac_ML\Assignments\Assignment_3\config\config.yaml loaded successfully]
[2025-11-06 23:42:59,354: INFO: common: yaml file: D:\Cdac_ML\Assignments\Assignment_3\params.yaml loaded successfully]
[2025-11-06 23:42:59,354: INFO: common: yaml file: D:\Cdac_ML\Assignments\Assignment_3\schema.yaml loaded successfully]
[2025-11-06 23:42:59,354: INFO: common: created directory at: artifacts]
[2025-11-06 23:42:59,354: INFO: common: created directory at: artifacts/model_trainer]
